<a href="https://colab.research.google.com/github/TongyuZ/GenAI_Final/blob/main/Surrogate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import random

ds = load_dataset("zeroshot/twitter-financial-news-sentiment")

def sample_and_split_data(dataset, num_per_label=1000, train_split=0.8):
    sampled_data = {label: [] for label in range(3)}
    train_data, test_data = [], []

    for example in dataset:
        label = example['label']
        if len(sampled_data[label]) < num_per_label:
            sampled_data[label].append(example)

    for label, examples in sampled_data.items():
        random.shuffle(examples)
        split_idx = int(len(examples) * train_split)
        train_data.extend(examples[:split_idx])
        test_data.extend(examples[split_idx:])

    return train_data, test_data

train_data, test_data = sample_and_split_data(ds['train'], num_per_label=1000)

from datasets import Dataset
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)


README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

# load in the model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=50,
    save_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-ebd9427f5d4b>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")


Step,Training Loss,Validation Loss


('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json',
 './fine_tuned_distilbert/tokenizer.json')

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

from sklearn.metrics import classification_report
print(classification_report(true_labels, predicted_labels, target_names=["Bearish (0)", "Bullish (1)", "Neutral (2)"]))


              precision    recall  f1-score   support

 Bearish (0)       0.80      0.81      0.81       200
 Bullish (1)       0.78      0.79      0.78       200
 Neutral (2)       0.79      0.78      0.78       200

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600



In [ ]:
import torch

In [ ]:
model_path = "./fine_tuned_distilbert"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
import nltk
from nltk.corpus import words

In [ ]:
nltk.download('words')

# Get words from nltk
vocabulary = words.words()

vocabulary = vocabulary[:5000]

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
print(f"Vocabulary size: {len(vocabulary)}")
print(f"Sample words from vocabulary: {vocabulary[:10]}")


Vocabulary size: 5000
Sample words from vocabulary: ['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron']


In [ ]:
def find_adversarial_suffix_nltk(text, true_label, vocabulary, max_suffix_length=5):

    current_text = text
    best_suffix = ""

    for _ in range(max_suffix_length):
        best_candidate = None
        best_error_score = float("-inf")

        for token in vocabulary:

            modified_text = current_text + " " + token


            inputs = tokenizer(modified_text, return_tensors="pt", truncation=True, max_length=512).to(device)


            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.softmax(logits, dim=-1).cpu().numpy()[0]

            predicted_label = probabilities.argmax()
            predicted_score = probabilities[predicted_label]

            if predicted_label != true_label and predicted_score > best_error_score:
                best_error_score = predicted_score
                best_candidate = token

        if best_candidate is None:
            break

        best_suffix += " " + best_candidate
        current_text += " " + best_candidate

    return best_suffix.strip()

In [ ]:
def display_predictions(original_text, adversarial_suffix, true_label, model, tokenizer):
    """
    Display predictions for original text and adversarial text.
    """

    inputs = tokenizer(original_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    original_label = probabilities.argmax()
    original_prob = probabilities[original_label]

    adversarial_text = original_text + " " + adversarial_suffix
    inputs = tokenizer(adversarial_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    adversarial_label = probabilities.argmax()
    adversarial_prob = probabilities[adversarial_label]

    print(f"Original Text: {original_text}")
    print(f"Original Prediction: LABEL_{original_label}, Probability: {original_prob:.4f}")
    print(f"Adversarial Suffix: {adversarial_suffix}")
    print(f"Adversarial Text: {adversarial_text}")
    print(f"Adversarial Prediction: LABEL_{adversarial_label}, Probability: {adversarial_prob:.4f}")
    print("-" * 50)


In [ ]:
input_texts = [
    "The stock market is crashing, investors are panicking.",
    "The market is hitting new all-time highs, optimism is everywhere."
]
true_labels = [0, 1]

for text, true_label in zip(input_texts, true_labels):
    adversarial_suffix = find_adversarial_suffix_nltk(text, true_label, vocabulary, max_suffix_length=5)
    display_predictions(text, adversarial_suffix, true_label, model, tokenizer)

Original Text: The stock market is crashing, investors are panicking.
Original Prediction: LABEL_0, Probability: 0.9419
Adversarial Suffix: 
Adversarial Text: The stock market is crashing, investors are panicking. 
Adversarial Prediction: LABEL_0, Probability: 0.9419
--------------------------------------------------
Original Text: The market is hitting new all-time highs, optimism is everywhere.
Original Prediction: LABEL_1, Probability: 0.9025
Adversarial Suffix: abuser acridly across affected Acarus
Adversarial Text: The market is hitting new all-time highs, optimism is everywhere. abuser acridly across affected Acarus
Adversarial Prediction: LABEL_0, Probability: 0.9300
--------------------------------------------------
Original Text: The market is stable with no significant movement.
Original Prediction: LABEL_1, Probability: 0.7901
Adversarial Suffix: advancing admitted additional afterswell Adamitical
Adversarial Text: The market is stable with no significant movement. advancing

In [ ]:
def sample_sentences(dataset, label, n_samples=100):

    sentences = [example["text"] for example in dataset if example["label"] == label]
    return random.sample(sentences, n_samples)

train_data = ds["train"]
input_texts_0 = sample_sentences(train_data, label=0, n_samples=100)
input_texts_1 = sample_sentences(train_data, label=1, n_samples=100)
input_texts = input_texts_0 + input_texts_1
true_labels = [0] * 100 + [1] * 100

In [ ]:
vocabulary = [
    "The market is growing rapidly even though",
    "This trend looks very risky even though",
    "Investors are optimistic even though",
    "Market decline is evident even though",
    "Bullish trends are dominating even though",
    "Bearish sentiment is rising even though",
    "Stable market is expected even though",
    "Volatility is increasing even though",
    "The market shows strong recovery even though",
    "This is a weak performance even though"
]

In [ ]:
def find_adversarial_suffix_with_vocab(text, true_label, vocab_sentences):

    best_candidate = None
    best_error_score = float("-inf")

    for sentence in vocab_sentences:
        modified_text = text + " " + sentence
        inputs = tokenizer(modified_text, return_tensors="pt", truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        predicted_label = probabilities.argmax()
        predicted_score = probabilities[0, predicted_label]

        # If the label is altered and has the higher probability
        if predicted_label != true_label and predicted_score > best_error_score:
            best_error_score = predicted_score
            best_candidate = sentence

    return best_candidate if best_candidate else ""

In [ ]:
def display_predictions(original_text, adversarial_suffix, model, tokenizer):

    inputs = tokenizer(original_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        original_probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
    original_label = original_probabilities.argmax()
    original_prob = original_probabilities[0, original_label]

    adversarial_text = original_text + " " + adversarial_suffix
    inputs = tokenizer(adversarial_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        adversarial_probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
    adversarial_label = adversarial_probabilities.argmax()
    adversarial_prob = adversarial_probabilities[0, adversarial_label]

    print(f"Original Text: {original_text}")
    print(f"Original Prediction: LABEL_{original_label}, Probabilities: {original_probabilities}")
    print(f"Adversarial Suffix: {adversarial_suffix}")
    print(f"Adversarial Text: {adversarial_text}")
    print(f"Adversarial Prediction: LABEL_{adversarial_label}, Probabilities: {adversarial_probabilities}")
    print("-" * 50)


for text, true_label in zip(input_texts, true_labels):
    adversarial_suffix = find_adversarial_suffix_with_vocab(text, true_label, vocabulary)
    display_predictions(text, adversarial_suffix, model, tokenizer)


Original Text: Clovis Oncology stock price target cut to $8 from $12 at Evercore ISI
Original Prediction: LABEL_0, Probabilities: [[0.9595499  0.01947849 0.0209715 ]]
Adversarial Suffix: 
Adversarial Text: Clovis Oncology stock price target cut to $8 from $12 at Evercore ISI 
Adversarial Prediction: LABEL_0, Probabilities: [[0.9595499  0.01947849 0.0209715 ]]
--------------------------------------------------
Original Text: $TSLA - Tesla's New Pickup Is Ugly, Sell; Also Week-End Rally Pattern Holding But Weakly. Get more updates here:… https://t.co/5w5TEy85xb
Original Prediction: LABEL_0, Probabilities: [[0.9286073  0.03459749 0.03679525]]
Adversarial Suffix: 
Adversarial Text: $TSLA - Tesla's New Pickup Is Ugly, Sell; Also Week-End Rally Pattern Holding But Weakly. Get more updates here:… https://t.co/5w5TEy85xb 
Adversarial Prediction: LABEL_0, Probabilities: [[0.9286073  0.03459749 0.03679525]]
--------------------------------------------------
Original Text: The global oil market i

In [ ]:
for text, true_label in zip(input_texts, true_labels):
    adversarial_suffix = find_adversarial_suffix_with_vocab(text, true_label, vocabulary)
    display_predictions(text, adversarial_suffix, model, tokenizer)


Original Text: Aurora Cannabis’s stock in decline; CEO is out, cost cuts announced
Original Prediction: LABEL_0, Probabilities: [[0.95833105 0.02177288 0.01989599]]
Adversarial Suffix: 
Adversarial Text: Aurora Cannabis’s stock in decline; CEO is out, cost cuts announced 
Adversarial Prediction: LABEL_0, Probabilities: [[0.95833105 0.02177288 0.01989599]]
--------------------------------------------------
Original Text: La Jolla Pharma to reassess development of LJPC-401
Original Prediction: LABEL_2, Probabilities: [[0.03977733 0.8953724  0.06485026]]
Adversarial Suffix: Bearish sentiment is rising.
Adversarial Text: La Jolla Pharma to reassess development of LJPC-401 Bearish sentiment is rising.
Adversarial Prediction: LABEL_1, Probabilities: [[0.03977733 0.8953724  0.06485026]]
--------------------------------------------------
Original Text: Children's Place downgraded as e-commerce puts margins at risk
Original Prediction: LABEL_0, Probabilities: [[0.9588425  0.01958273 0.02157481]